# Number of Hidden Neurons in 2017

In [28]:
from Gaugi.tex import *
import numpy as np
import array
import os
from functools import reduce
from pprint import pprint
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
etbins = [15, 20, 30, 40, 50, 1000000]
etabins = [0.0, 0.8, 1.37, 1.54, 2.50]
#etabins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.50]

In [36]:
def get_models( location , etbins, etabins, remove_last=True):
    from tensorflow.keras.models import Model, model_from_json
    from tensorflow.keras.layers import Activation
    import json
    models = [[ None for _ in range(len(etabins)-1)] for __ in range(len(etbins)-1)]
    for et_bin in range(len(etbins)-1):
        for eta_bin in range(len(etabins)-1):
            d_tuned = {}
            path=location.format(ET=et_bin,ETA=3 if eta_bin==4 else eta_bin)
            #print(path)
            with open(path+'.json', 'r') as json_file:
                model = model_from_json(json_file.read())
                # load weights into new model
                model.load_weights(path+".h5")
                # since the production version has no tansig fucntion as default. Let's include it
                # for study
                model.add(Activation('tanh'))
                new_model = Model(model.inputs, model.layers[-2].output) if remove_last else model
                d_tuned['model']    = new_model
                d_tuned['etBin']    = [etbins[et_bin], etbins[et_bin+1]]
                d_tuned['etaBin']   = [etabins[eta_bin], etabins[eta_bin+1]]
                d_tuned['etBinIdx'] = et_bin
                d_tuned['etaBinIdx']= eta_bin
                models[et_bin][eta_bin] = d_tuned
                
    # return the models's matrix          
    return models

model_path = '/Volumes/castor/tuning_data/Zee/production/TrigL2_20170505_v6/models'
model_path+='/mc15_13TeV.423300.Zee_probes_lhmedium.423300.JF17_Truth.model_v6.electronTight.et{ET}_eta{ETA}'
best_models = get_models(model_path, etbins, etabins,remove_last=True)

In [58]:
def make_table(best_models, etbins, etabins, title, output):
    
    etbins_str = []; etabins_str = []
    for etBinIdx in range(len(etbins)-1):
        etbin = (etbins[etBinIdx], etbins[etBinIdx+1])
        if etbin[1] > 100 :
            etbins_str.append( r'$E_{T}\text{[GeV]} > %d$' % etbin[0])
        else:
            etbins_str.append(  r'$%d < E_{T} \text{[Gev]}<%d$'%etbin )

    # Create Latex eta bins
    for etaBinIdx in range( len(etabins)-1 ):
        etabin = (etabins[etaBinIdx], etabins[etaBinIdx+1])
        etabins_str.append( r'$%.2f<\eta<%.2f$'%etabin )
    
    green = '\cellcolor[HTML]{9AFF99}'

    with BeamerTexReportTemplate1( theme = 'Berlin'
                                 , _toPDF = True
                                 , title = title
                                 , outputFile = output
                                 , font = 'structurebold' ):
        
        lines1 = []
        lines1 += [ HLine(_contextManaged = False) ]
        lines1 += [ HLine(_contextManaged = False) ]
        lines1 += [ TableLine( columns = ['',''] + reduce(lambda x,y: x+y,[[s] for s in etbins_str]), _contextManaged = False ) ]
        #lines1 += [ HLine(_contextManaged = False) ]
        #lines1 += [ TableLine( columns = ['',''] + reduce(lambda x,y: x+y,[['Units'] for _ in etbins_str]), _contextManaged = False ) ]
        lines1 += [ HLine(_contextManaged = False) ]
        layers = ['Input','Hidden','Output']
        for etaBinIdx in range(len(etabins)-1):
            for idx, layer in enumerate( layers ):
                values = []
                for etBinIdx in range(len(etbins)-1):
                    color = green if idx==1 else ""
                    if idx==0:
                        n=100
                    elif idx==1:
                        n = best_models[etBinIdx][etaBinIdx]['model'].layers[1].bias.shape[0]
                    else:
                        n = 1
                    values.append( color+str(n) )   
                    
                
                if idx > 0:
                    lines1 += [ TableLine( columns = ['', layer] + values   , _contextManaged = False ) ]
                else:
                    lines1 += [ TableLine( columns = ['\multirow{%d}{*}{'%(len(layers))+etabins_str[etaBinIdx]+'}', layer] + values   ,
                                                _contextManaged = False ) ]
                    #lines1 += [ TableLine( columns = ['', layer] + values   , _contextManaged = False ) ]
            lines1 += [ HLine(_contextManaged = False) ]
        lines1 += [ HLine(_contextManaged = False) ]
        
        # Create all tables into the PDF Latex
        with BeamerSlide( title = "Number of neuron in hidden layer for 2017 operation"  ):
            with Table( caption = "Number of neuron in hidden layer for 2017 operation.") as table:
                with ResizeBox( size = 1. ) as rb:
                    with Tabular( columns = 'lcc' + 'c' * len(etbins_str) ) as tabular:
                        tabular = tabular
                        for line in lines1:
                            if isinstance(line, TableLine):
                                tabular += line
                            else:
                                TableLine(line, rounding = None)
        
        
make_table(best_models, etbins, etabins, 'Number of neurons (hidden layer) for 2017 (v6)', 'number_of_neurons_for_v6.pdf')

2021-01-15 14:17:49,444 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file number_of_neurons_for_v6.pdf latex code...
